## Aula 06 - Filtragem Híbrida - Exercícios

In [2]:
import pandas as pd
import numpy as np

### Importar base de dados

In [3]:
import wget
!python3 -m wget https://github.com/mmanzato/MBABigData/raw/master/ml-20m-compact.tar.gz
!tar -xvzf ml-20m-compact.tar.gz

100% [....................................................] 65019041 / 65019041
Saved under ml-20m-compact.tar (1).gz
dataset/
dataset/tags_sample.csv
dataset/._.DS_Store
dataset/.DS_Store
dataset/movies_sample.csv
dataset/._genome-tags.csv
dataset/genome-tags.csv
dataset/._ml-youtube.csv
dataset/ml-youtube.csv
dataset/._genome-scores.csv
dataset/genome-scores.csv
dataset/ratings_sample.csv


In [4]:
!python3 -m wget https://github.com/mmanzato/MBABigData/raw/master/ml-20m-features.tar.gz
! tar -xvzf ml-20m-features.tar.gz

100% [......................................................] 5996435 / 5996435
Saved under ml-20m-features.tar (1).gz
features/
features/._m4infus_max_histogram_300_sn.arq
features/m4infus_max_histogram_300_sn.arq
features/._mm_avg_histogram_100_sn.arq
features/mm_avg_histogram_100_sn.arq
features/._visual_histogram_100_sn.arq
features/visual_histogram_100_sn.arq
features/._visual_histogram_50_sn.arq
features/visual_histogram_50_sn.arq
features/._aural_histogram_50.arq
features/aural_histogram_50.arq
features/._mm_max_histogram_300.arq
features/mm_max_histogram_300.arq
features/._m4infus_max_histogram_50.arq
features/m4infus_max_histogram_50.arq
features/._mm_max_histogram_100.arq
features/mm_max_histogram_100.arq
features/._mm_max_histogram_50_sn.arq
features/mm_max_histogram_50_sn.arq
features/._visual_histogram_100.arq
features/visual_histogram_100.arq
features/._visual_histogram_300.arq
features/visual_histogram_300.arq
features/._aural_histogram_100_sn.arq
features/aural_histogra

### Preparação dos dados

In [5]:
movies = pd.read_csv('./dataset/movies_sample.csv')
ratings = pd.read_csv('./dataset/ratings_sample.csv')
df = ratings[['userId', 'movieId', 'rating']]
df = df.merge(movies[['movieId', 'title']])
df

,userId,movieId,rating,title
0,11,7481,5.0,Enemy Mine (1985)
1,11,1046,4.5,Beautiful Thing (1996)
2,11,616,4.0,"Aristocats, The (1970)"
3,11,3535,2.0,American Psycho (2000)
4,11,5669,5.0,Bowling for Columbine (2002)
...,...,...,...,...
190616,138493,288,5.0,Natural Born Killers (1994)
190617,138493,1748,5.0,Dark City (1998)
190618,138493,616,4.0,"Aristocats, The (1970)"
190619,138493,1597,4.5,Conspiracy Theory (1997)


In [6]:
movies_genres = movies.drop('genres', axis=1).join(movies.genres.str.split('|', expand=True)
             .stack().reset_index(drop=True, level=1).rename('genre'))
movies_genres.head()

,movieId,title,genre
0,30,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,Crime
0,30,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,Drama
1,31,Dangerous Minds (1995),Drama
2,37,Across the Sea of Time (1995),Documentary
2,37,Across the Sea of Time (1995),IMAX


In [7]:
movies_tags = pd.read_csv('./dataset/tags_sample.csv')
movies_tags.head()

,userId,movieId,tag,timestamp_y
0,279,916,Gregory Peck,1329962459
1,279,916,need to own,1329962471
2,279,916,romantic comedy,1329962476
3,279,916,Rome,1329962490
4,279,916,royalty,1329962474


In [8]:
map_users = {user: idx for idx, user in enumerate(df.userId.unique())}
map_items = {item: idx for idx, item in enumerate(df.movieId.unique())}

df['userId'] = df['userId'].map(map_users)
df['movieId'] = df['movieId'].map(map_items)

movies_tags['userId'] = movies_tags['userId'].map(map_users)
movies_tags['movieId'] = movies_tags['movieId'].map(map_items)
movies_tags.dropna(inplace=True)
movies_tags['movieId'] = movies_tags.movieId.astype(int)
movies_tags['userId'] = movies_tags.userId.astype(int)

movies_genres['movieId'] = movies_genres['movieId'].map(map_items)
movies_genres.dropna(inplace=True)
movies_genres['movieId'] = movies_genres.movieId.astype(int)

map_title = {}
for _, row in df.iterrows():
    map_title[row.movieId] = row.title


### Divisão da base em treino/teste

In [9]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=.2, random_state=2)
train.to_csv('train.dat', index=False, header=False, sep='\t')
test.to_csv('test.dat', index=False, header=False, sep='\t')

### Download de dados multimidia

In [10]:
import pickle

with open('./features/visual_histogram_50.arq', 'rb') as arq_visualHistograms:
    visualHistograms = pickle.load(arq_visualHistograms)

with open('./features/aural_histogram_50.arq', 'rb') as arq_auralHistograms:
    auralHistograms = pickle.load(arq_auralHistograms)

In [11]:
movies

,movieId,title,genres
0,30,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,Crime|Drama
1,31,Dangerous Minds (1995),Drama
2,37,Across the Sea of Time (1995),Documentary|IMAX
3,161,Crimson Tide (1995),Drama|Thriller|War
4,193,Showgirls (1995),Drama
...,...,...,...
428,129857,Bright Days Ahead (2013),Drama|Romance
429,130344,Coast of Death (2013),Children|Documentary|Drama
430,130408,A Rumor Of War (1980),(no genres listed)
431,130510,Berserk: The Golden Age Arc 3 - Descent (2013),Action|Animation|Fantasy


In [12]:
len(auralHistograms)

433

In [13]:
movies

,movieId,title,genres
0,30,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,Crime|Drama
1,31,Dangerous Minds (1995),Drama
2,37,Across the Sea of Time (1995),Documentary|IMAX
3,161,Crimson Tide (1995),Drama|Thriller|War
4,193,Showgirls (1995),Drama
...,...,...,...
428,129857,Bright Days Ahead (2013),Drama|Romance
429,130344,Coast of Death (2013),Children|Documentary|Drama
430,130408,A Rumor Of War (1980),(no genres listed)
431,130510,Berserk: The Golden Age Arc 3 - Descent (2013),Action|Animation|Fantasy


In [15]:
tag_by_movie = movies_tags.groupby("movieId").agg({"tag": list}).reset_index()

## ***Exercício 01:*** Implemente uma hibridização monolítica/combinação usando a seguinte heurística:
- Uso do algoritmo ItemAtributeKNN, sendo a hibridização feita no cálculo das similaridades entre os itens.
- Se a quantidade de usuários que avaliaram ambos os itens for maior que um limiar L1, calcule a similaridade entre esses itens usando cosseno aplicado à representação baseada em notas.
- Caso contrário, calcule a similaridade entre os itens usando tags, características visuais e características aurais. Pondere cada uma das modalidades via pesos passados por parâmetro. 

Compare os resultados do algoritmo híbrido com as versões isoladas do mesmo algoritmo.

In [168]:
def get_tags(tag_by_movie_df, movie_id):
    tags = tag_by_movie_df[tag_by_movie_df.movieId == movie_id].tag.tolist()
    if (len(tags) > 0):
        return tags[0]
    else:
        return []
    

In [38]:
def users_who_rated(item, df):
    return set(df[df["movieId"] == item].userId.tolist())

In [55]:
def get_movies_ratings(movie_id, user_list, df):
    users_ratings = df[df["userId"].isin(user_list)]
    return users_ratings[users_ratings["movieId"] == movie_id].rating.tolist()

In [172]:
def jaccard(list1, list2):
    union_s = len(set(list1).union(set(list2)))
    inter_s = len(set(list1) & set(list2))

    if (union_s > 0):
        return inter_s/union_s
    else:
        return 0

In [122]:
def multimedia_sim(i,j, visual_w=0.6, aural_w=0.2, tag_w=0.2):
    visual_sim = cossine_sim(visualHistograms[i], visualHistograms[j])
    aural_sim = cossine_sim(auralHistograms[i], auralHistograms[j])

    tag_sim = jaccard(get_tags(tag_by_movie, i), get_tags(tag_by_movie, j))
    return visual_w * visual_sim + aural_w * aural_sim + tag_w * tag_sim

In [190]:
from itertools import combinations

def build_hybrid_rec_sim_matrix(df, L1=10):
    movies_ids = sorted(df.movieId.unique())
    sim_matrix = []
    for i, j in combinations(movies_ids, 2):
            inter_users = get_users_who_rated(i, j)
            n_inter_users = len(inter_users)
            if (n_inter_users >= L1):
                vec_1 = get_movies_ratings(i, inter_users, df)
                vec_2 = get_movies_ratings(j, inter_users, df)
                sim = cossine_sim(vec_1,vec_2)
            else:
                sim = multimedia_sim(i,j)
            sim_matrix.append([i, j, sim])
    return pd.DataFrame(sim_matrix, columns=['item1', 'item2', 'similarity'])

In [84]:
def get_users_who_rated(item1, item2):
    rated_1 = users_who_rated(item1, train)
    rated_2 = users_who_rated(item2, train)

    return list(rated_1 & rated_2)

In [90]:
from numpy.linalg import norm


def cossine_sim(a,b):
    numerator = np.dot(a,b)
    denominator = norm(a) *norm(b)
    if (denominator != 0):
        return numerator / denominator 
    else:
        return 0

In [23]:
def get_movies_rated_by_user(user_id, df):
    return df[df["userId"] == user_id].movieId.tolist()

In [191]:
%%time
sim_matrix = build_hybrid_rec_sim_matrix(train)

CPU times: user 3min 35s, sys: 54.6 ms, total: 3min 35s
Wall time: 3min 35s


In [192]:
sim_matrix

,item1,item2,similarity
0,0,1,0.958886
1,0,2,0.939329
2,0,3,0.939921
3,0,4,0.933063
4,0,5,0.946121
...,...,...,...
81805,413,415,0.154522
81806,413,416,0.313138
81807,414,415,0.426510
81808,414,416,0.303420


In [193]:
sim_matrix.to_csv('sim_matrix.dat', index=False, sep='\t', header=False)

In [202]:
tag_by_movie[["movieId", "tag"]].to_csv('items_genres.dat', index=False, sep='\t', header=False)

In [21]:
from caserec.recommenders.rating_prediction.item_attribute_knn import ItemAttributeKNN
from caserec.recommenders.rating_prediction.itemknn import ItemKNN

In [196]:
ItemAttributeKNN('train.dat', 'test.dat', similarity_file='sim_matrix.dat', k_neighbors=5, as_similar_first=True).compute()

[Case Recommender: Rating Prediction > Item Attribute KNN Algorithm]

train data:: 11090 users and 405 items (152496 interactions) | sparsity:: 96.60%
test data:: 10571 users and 331 items (38125 interactions) | sparsity:: 98.91%

training_time:: 4.603380 sec
prediction_time:: 0.201605 sec
Eval:: MAE: 0.667807 RMSE: 0.871053 


In [203]:
ItemAttributeKNN('train.dat', 'test.dat', metadata_file='items_genres.dat', similarity_metric="jaccard", k_neighbors=5, as_similar_first=True).compute()

[Case Recommender: Rating Prediction > Item Attribute KNN Algorithm]

train data:: 11090 users and 405 items (152496 interactions) | sparsity:: 96.60%
test data:: 10571 users and 331 items (38125 interactions) | sparsity:: 98.91%

training_time:: 6.172487 sec
>> metadata:: 231 items and 229 metadata (231 interactions) | sparsity:: 99.56%
prediction_time:: 0.538390 sec
Eval:: MAE: 0.931913 RMSE: 1.184732 


In [199]:
ItemKNN('train.dat', 'test.dat', k_neighbors=5, as_similar_first=True).compute()

[Case Recommender: Rating Prediction > ItemKNN Algorithm]

train data:: 11090 users and 405 items (152496 interactions) | sparsity:: 96.60%
test data:: 10571 users and 331 items (38125 interactions) | sparsity:: 98.91%

training_time:: 4.465057 sec
prediction_time:: 1.053713 sec
Eval:: MAE: 0.72067 RMSE: 0.952334 


Por padrão, itemKNN usa distancia de cossenos como metrica de similaridade. 

Note que utilizando os mesmos conjuntos de dados e mesmo numero de vizinhos, o uso de metadados aparentemente auxiliou a versão hibrida: o MAE ( e, em consequencia, o RMSE) da versão hibrida é menor se comparada à versão puramente colaborativa.

A conclusão é a mesma para a variante que utiliza metadados isoladamente (por simplicidade, apenas avaliei o desempenho com generos  como metadado)

## ***Exercício 02:*** 

Vamos implementar um recomendador híbrido canalizado em cascata, no cenário de ranqueamento. A ideia é que um primeiro algoritmo gere uma lista C1 de N=50 itens candidatos à recomendação para cada usuário. Em seguida, um outro recomendador irá gerar uma outra lista C2 também de N=50 itens candidatos à rcomendação para cada usuário. Por fim, o ranking final será a intersecção entre C1 e C2, sendo o score de cada itens formado pela média aritmética dos scores de cada lista. Avalie o desempenho.

Dica 1: utilize o parâmetro rank_length disponível nos algoritmos de ranqueamento do CaseRecommender para especificar o tamanho N de recomendações para cada usuário.

Dica 2: você pode gravar num arquivo os rankings gerados por um algoritmo para cada usuário especificando o nome do arquivo no parâmetro output_file.

Dica 3: consulte a Aula 04 que contém algumas métricas de avaliação de ranqueamento. Como você irá gerar o ranking final externamente ao CaseRecommender, será necessário avaliá-lo usando funções próprias.

In [42]:
from caserec.recommenders.item_recommendation.bprmf import BprMF
from caserec.recommenders.item_recommendation.itemknn import ItemKNN

In [24]:
BprMF('train.dat', 'test.dat', 'first_pass.dat', factors=10, rank_length=50).compute()

[Case Recommender: Item Recommendation > BPRMF]

train data:: 11090 users and 405 items (152496 interactions) | sparsity:: 96.60%
test data:: 10571 users and 331 items (38125 interactions) | sparsity:: 98.91%

training_time:: 119.875303 sec
prediction_time:: 2.808603 sec


Eval:: PREC@1: 0.357582 PREC@3: 0.271718 PREC@5: 0.232561 PREC@10: 0.177959 RECALL@1: 0.11457 RECALL@3: 0.251705 RECALL@5: 0.35281 RECALL@10: 0.524745 MAP@1: 0.357582 MAP@3: 0.456217 MAP@5: 0.465186 MAP@10: 0.442534 NDCG@1: 0.357582 NDCG@3: 0.548851 NDCG@5: 0.574998 NDCG@10: 0.574218 


In [25]:
recs_bpmrf = pd.read_csv('./first_pass.dat', sep='\t', names=['userId', 'movieId', 'score'])

In [26]:
recs_bpmrf

,userId,movieId,score
0,0,19,7.660311
1,0,18,6.913694
2,0,46,6.873506
3,0,108,6.759038
4,0,5,6.674600
...,...,...,...
554495,11089,46,3.324016
554496,11089,55,3.288072
554497,11089,56,3.242554
554498,11089,38,3.221053


In [43]:
ItemKNN('train.dat', 'test.dat', rank_length=50, output_file="second_pass.dat", k_neighbors=5, as_similar_first=True).compute()

[Case Recommender: Item Recommendation > ItemKNN Algorithm]

train data:: 11090 users and 405 items (152496 interactions) | sparsity:: 96.60%
test data:: 10571 users and 331 items (38125 interactions) | sparsity:: 98.91%

training_time:: 0.979557 sec
prediction_time:: 28.812758 sec


Eval:: PREC@1: 0.338 PREC@3: 0.262479 PREC@5: 0.223214 PREC@10: 0.17215 RECALL@1: 0.109669 RECALL@3: 0.244771 RECALL@5: 0.340091 RECALL@10: 0.510392 MAP@1: 0.338 MAP@3: 0.441585 MAP@5: 0.45211 MAP@10: 0.432388 NDCG@1: 0.338 NDCG@3: 0.538688 NDCG@5: 0.564177 NDCG@10: 0.568101 


In [237]:
def intersect_and_average(list1, list2):
    dict1 = {item1: item2 for item1, item2 in list1}
    dict2 = {item1: item2 for item1, item2 in list2}
    
    common_keys = set(dict1.keys()).intersection(dict2.keys())
    result = [(key, (dict1[key] + dict2[key]) / 2) for key in common_keys]
    
    return result


In [188]:
def sort_recs(rec):
    sorted_rec_with_score = sorted(rec, key=lambda x: x[1], reverse=True)
    return [l[0] for l in sorted_rec_with_score]

In [234]:
def preprocess_recommendation(rec_df):
    grouped = rec_df.groupby("userId").agg({"movieId": list, "score": list})
    grouped["movie_with_rating"] = grouped.apply(lambda r: create_tuples(r), axis=1)
    return grouped.reset_index()[["userId", "movie_with_rating"]]

In [296]:
def merge_recommendations(recdf_1, recdf2):
    merged = pd.concat([recs_bpmrf_processed, recs_itemKNN_processed], axis=1).drop(columns=["userId_itemknn"])
    merged["merged_list"] = merged.apply(lambda r: intersect_and_average(r.movie_with_rating_bpmrf, r.movie_with_rating_itemknn), axis=1)
    merged_clean = merged.drop(columns=["movie_with_rating_bpmrf", "movie_with_rating_itemknn"])
    merged_clean["recs"] = merged_clean.apply(lambda r: sort_recs(r.merged_list), axis=1)

    
    return merged_clean.drop(columns=["merged_list"]).rename(columns={'userId_bpmrf': 'userId'})

In [44]:
recs_itemKNN = pd.read_csv('./second_pass.dat', sep='\t', names=['userId', 'movieId', 'score'])

In [226]:
def create_tuples(row):
    return list(zip(row['movieId'], row['score']))

In [249]:
recs_bpmrf_processed =  preprocess_recommendation(recs_bpmrf).add_suffix('_bpmrf')

In [251]:
recs_itemKNN_processed = preprocess_recommendation(recs_itemKNN).add_suffix('_itemknn')

In [254]:
teste=pd.concat([recs_bpmrf_processed, recs_itemKNN_processed], axis=1).drop(columns=["userId_itemknn"])

In [267]:
teste["merged_list"] = teste.apply(lambda r: intersect_and_average(r.movie_with_rating_bpmrf, r.movie_with_rating_itemknn), axis=1)

In [268]:
teste2 = teste.drop(columns=["movie_with_rating_bpmrf", "movie_with_rating_itemknn"])

In [270]:
teste2["sorted_recs"] = teste2.apply(lambda r: sort_recs(r.merged_list), axis=1)

In [275]:
teste2.rename(columns={"userId_bpmrf": "ble"})

,ble,merged_list,sorted_recs
0,0,"[(0, 2.012203), (131, 2.660602), (5, 3.7659905...","[19, 18, 5, 108, 22, 46, 30, 39, 17, 80, 57, 3..."
1,1,"[(2, 1.663777), (4, 3.730434), (5, 3.217233000...","[19, 10, 14, 8, 4, 20, 28, 33, 13, 5, 31, 30, ..."
2,2,"[(1, 1.3718670000000002), (2, 2.68317399999999...","[14, 10, 33, 21, 4, 3, 31, 61, 5, 34, 56, 30, ..."
3,3,"[(2, 2.23869), (4, 3.5521145), (5, 3.525597), ...","[12, 22, 14, 18, 33, 4, 16, 5, 23, 28, 57, 20,..."
4,4,"[(0, 1.7577340000000001), (2, 3.006087), (3, 3...","[17, 33, 21, 3, 22, 28, 5, 26, 30, 13, 31, 57,..."
...,...,...,...
11085,11085,"[(6, 2.681425), (8, 3.120589), (10, 3.783667),...","[19, 12, 18, 10, 22, 28, 80, 81, 39, 46, 8, 20..."
11086,11086,"[(2, 3.1472995000000004), (4, 3.2869845), (5, ...","[12, 83, 5, 8, 4, 23, 22, 2, 10, 52, 43, 34, 3..."
11087,11087,"[(0, 3.2701765000000003), (129, 2.6652285), (3...","[6, 57, 4, 84, 10, 16, 196, 36, 38, 18, 3, 33,..."
11088,11088,"[(2, 2.5689845), (3, 3.379664), (4, 3.0791), (...","[19, 17, 21, 8, 22, 10, 28, 20, 64, 3, 29, 15,..."


In [298]:
merged_recs = merge_recommendations(recs_bpmrf_processed, recs_itemKNN_processed)

In [299]:
merged_recs

,userId,recs
0,0,"[19, 18, 5, 108, 22, 46, 30, 39, 17, 80, 57, 3..."
1,1,"[19, 10, 14, 8, 4, 20, 28, 33, 13, 5, 31, 30, ..."
2,2,"[14, 10, 33, 21, 4, 3, 31, 61, 5, 34, 56, 30, ..."
3,3,"[12, 22, 14, 18, 33, 4, 16, 5, 23, 28, 57, 20,..."
4,4,"[17, 33, 21, 3, 22, 28, 5, 26, 30, 13, 31, 57,..."
...,...,...
11085,11085,"[19, 12, 18, 10, 22, 28, 80, 81, 39, 46, 8, 20..."
11086,11086,"[12, 83, 5, 8, 4, 23, 22, 2, 10, 52, 43, 34, 3..."
11087,11087,"[6, 57, 4, 84, 10, 16, 196, 36, 38, 18, 3, 33,..."
11088,11088,"[19, 17, 21, 8, 22, 10, 28, 20, 64, 3, 29, 15,..."
